# GEORREFERENCIAMENTO OVP-DH
## Jupyter Notebook compartilhado

In [34]:
import numpy as np
import pandas as pd
import json
import ipywidgets
import folium
from folium import plugins
from folium.plugins import MarkerCluster
import psycopg2
from sqlalchemy import create_engine
from getpass import getpass

### Abrindo dataset

In [52]:
#abrindo o arquivo csv e eliminando a coluna sem dados
data = pd.read_csv('../data/amostra_data_geo_bo2016.csv').drop(columns=['Unnamed: 0'])
#lendo as cinco primeiras linhas
data.head()

,NUM_BO,DELEGACIA,RUBRICA,MES,LATITUDE,LONGITUDE
0,13,85º D.P. JARDIM MIRNA,Roubo,1,-23.770856,-46.675564
1,185,06º D.P. OSASCO,Furto,1,-23.544059,-46.783202
2,63771,DELEGACIA ELETRONICA,Furto,1,-23.477720,-46.686221
3,117980,DELEGACIA ELETRONICA,Roubo,1,-23.662874,-46.729696
4,55,01º D.P. DIADEMA,Lesão envolvendo veículos,1,-23.712553,-46.606715


## Construindo o mapa com marcadores

### Definindo 'natureza_lesao' como objetivo de pesquisa

In [53]:
# Visualizando a média da latitudes e longitudes das ocorrências
print("latitude:", data['LATITUDE'].mean())
print("longitude:", data['LONGITUDE'].mean())

latitude: -23.573094752577116
longitude: -46.618737618128854


In [88]:
# Salvando a média da latitudes e longitudes das ocorrências
latmean = data['LATITUDE'].mean()
lonmean = data['LONGITUDE'].mean()

# Criando o mapa
mapa_marcadores = folium.Map(
    location = [latmean,lonmean],
    zoom_start = 9,
    tiles = 'OpenStreetMap')

# Visualizar criacao
mapa_marcadores

### Adicionando os marcadores

In [89]:
# qual e o total de linhas?
len(data)

1459

In [90]:
# data = data.sample(frac=0.2, random_state=42)

for lat, lon, rubrica in zip(data['LATITUDE'],data['LONGITUDE'],data['RUBRICA']):
    folium.Marker(location = [lat,lon],
                  popup = rubrica,
                  icon = folium.Icon(color = 'red')).add_to(mapa_marcadores)

### Visualizando o mapa

In [91]:
mapa_marcadores

### Atribuindo cor segundo o tipo de lesao

In [92]:
# lista de lesoes
data['RUBRICA'].unique()

array(['Roubo', 'Furto', 'Lesão envolvendo veículos', 'Lesão corporal',
       'Homicídio doloso', 'Crime sexual', 'Crimes envolvendo drogas',
       'Homicídio envolvendo veículos'], dtype=object)

In [93]:
# criando dicionario para receferencia de cores
dic_nl = {'Roubo': 'darkgreen', 
          'Furto': 'lightgreen', 
          'Lesão envolvendo veículos': 'red', 
          'Lesão corporal': 'blue',
          'Homicídio doloso': 'black', 
          'Crime sexual': 'grey',
          'Crimes envolvendo drogas': 'purple',
          'Homicídio envolvendo veículos': 'deepred',}

In [94]:
# precisamos agora refazer o mapa, considerando as cores
mapa_marcadores = folium.Map(
    location = [latmean,lonmean],
    zoom_start = 9,
    tiles = 'OpenStreetMap')

for lat, lon, rubrica in zip(data['LATITUDE'],data['LONGITUDE'],data['RUBRICA']):
    folium.Marker(location = [lat,lon],
                  popup = rubrica,
                  icon = folium.Icon(color = dic_nl[rubrica])).add_to(mapa_marcadores)

/home/nacholez/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: color argument of Icon should be one of: {'purple', 'green', 'cadetblue', 'blue', 'lightred', 'darkpurple', 'darkgreen', 'darkblue', 'white', 'lightblue', 'gray', 'darkred', 'beige', 'lightgreen', 'orange', 'pink', 'lightgray', 'red', 'black'}.
  # Remove the CWD from sys.path while we load stuff.


In [95]:
# visualizando o mapa
mapa_marcadores

### Salvando o mapa

In [96]:
mapa_marcadores.save('../examples/mapa_marcadoreshtml')